In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

# Copiar para a pasta do Colab
#!cp -r /content/drive/MyDrive/2025.1/Projeto-Itau/CELEB-DF /content/celeb-df

In [2]:
# Instalar bibliotecas necessárias
#!pip install transformers torch torchvision opencv-python ffmpeg-python
import torch
import cv2
import numpy as np
import ffmpeg
from transformers import pipeline, AutoImageProcessor, AutoModelForVideoClassification

2025-03-21 23:29:55.273108: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-21 23:29:55.403047: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-21 23:29:55.434404: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 23:29:55.700450: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-21 23:29:57.383248: W tensorflow/compiler/tf2

In [3]:
# Função para extrair e redimensionar frames corretamente
def extract_and_resize_frames(video_path, num_frames=16, size=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frame_idxs = np.linspace(0, total_frames - 1, num_frames, dtype=int)  # Pega 16 frames uniformemente distribuídos

    for idx in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        if idx in frame_idxs:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Converter para RGB
            frame = cv2.resize(frame, size)  # Redimensionar para (224,224)
            frames.append(frame)

    cap.release()

    # Garantir exatamente 16 frames
    while len(frames) < num_frames:  # Se houver menos de 16 frames, repetir o último frame
        frames.append(frames[-1])

    return frames  # Retornar como lista de arrays NumPy

In [4]:
# Função para classificar um vídeo
def classify_video(video_path, processor, model, num_frames=32):    # Extrair e processar os frames corretamente
    frames = extract_and_resize_frames(video_path, num_frames=num_frames, size=(224, 224))
    inputs = processor(frames, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=-1).item()  # 1 se FAKE, 0 se REAL
    return prediction

In [9]:
import os

# Listar vídeos nos diretórios
fake_dir = '/home/flavio/Dev/Deepfake-Detection/dataset/celeb-df/Fake'
real_dir = '/home/flavio/Dev/Deepfake-Detection/dataset/celeb-df/Real'
fake_videos = [os.path.join(fake_dir, f) for f in os.listdir(fake_dir) if f.endswith(('.mp4', '.avi', '.mov'))]
real_videos = [os.path.join(real_dir, f) for f in os.listdir(real_dir) if f.endswith(('.mp4', '.avi', '.mov'))]

In [10]:
# Carregar modelo e processador
processor = AutoImageProcessor.from_pretrained("shylhy/videomae-large-finetuned-deepfake-subset")
model = AutoModelForVideoClassification.from_pretrained("shylhy/videomae-large-finetuned-deepfake-subset")

In [11]:
# Resultados
results = []
correct = 0
total = 0

In [ ]:
# Classificar vídeos falsos (esperado: 1)
for video_path in fake_videos:
    print(f"{total}. {video_path}")
    prediction = classify_video(video_path, processor, model, num_frames=32)  # Ajuste num_frames se necessário
    expected = 1  # FAKE
    is_correct = prediction == expected
    results.append({
        'video': os.path.basename(video_path),
        'predicted': 'FAKE' if prediction == 1 else 'REAL',
        'expected': 'FAKE',
        'correct': is_correct
    })
    correct += is_correct
    total += 1

['/home/flavio/Dev/Deepfake-Detection/dataset/celeb-df/Fake/id0_id2_0002.mp4', '/home/flavio/Dev/Deepfake-Detection/dataset/celeb-df/Fake/id0_id4_0002.mp4', '/home/flavio/Dev/Deepfake-Detection/dataset/celeb-df/Fake/id0_id9_0001.mp4', '/home/flavio/Dev/Deepfake-Detection/dataset/celeb-df/Fake/id0_id3_0003.mp4', '/home/flavio/Dev/Deepfake-Detection/dataset/celeb-df/Fake/id0_id1_0001.mp4', '/home/flavio/Dev/Deepfake-Detection/dataset/celeb-df/Fake/id0_id3_0008.mp4', '/home/flavio/Dev/Deepfake-Detection/dataset/celeb-df/Fake/id0_id3_0000.mp4', '/home/flavio/Dev/Deepfake-Detection/dataset/celeb-df/Fake/id0_id6_0002.mp4', '/home/flavio/Dev/Deepfake-Detection/dataset/celeb-df/Fake/id0_id3_0001.mp4', '/home/flavio/Dev/Deepfake-Detection/dataset/celeb-df/Fake/id0_id2_0007.mp4', '/home/flavio/Dev/Deepfake-Detection/dataset/celeb-df/Fake/id0_id3_0005.mp4', '/home/flavio/Dev/Deepfake-Detection/dataset/celeb-df/Fake/id0_id2_0005.mp4', '/home/flavio/Dev/Deepfake-Detection/dataset/celeb-df/Fake/id0_

KeyboardInterrupt: 

In [ ]:
# Classificar vídeos reais (esperado: 0)
for video_path in real_videos:
    prediction = classify_video(video_path, processor, model, num_frames=32)  # Ajuste num_frames se necessário
    expected = 0  # REAL
    is_correct = prediction == expected
    results.append({
        'video': os.path.basename(video_path),
        'predicted': 'FAKE' if prediction == 1 else 'REAL',
        'expected': 'REAL',
        'correct': is_correct
    })
    correct += is_correct
    total += 1

In [ ]:
# Exibir resultados
print("\nResultados da classificação:")
print(f"{'Vídeo':<30} {'Predito':<10} {'Esperado':<10} {'Correto':<10}")
print("-" * 60)
for result in results:
    print(f"{result['video']:<30} {result['predicted']:<10} {result['expected']:<10} {str(result['correct']):<10}")


Resultados da classificação:
Vídeo                          Predito    Esperado   Correto   
------------------------------------------------------------
id0_id6_0006.mp4               FAKE       FAKE       True      
id0_id3_0001.mp4               FAKE       FAKE       True      
id0_id3_0006.mp4               FAKE       FAKE       True      
id0_id6_0007.mp4               FAKE       FAKE       True      
id0_id3_0002.mp4               FAKE       FAKE       True      
id0_id1_0006.mp4               FAKE       FAKE       True      
id0_id2_0007.mp4               FAKE       FAKE       True      
id0_id1_0000.mp4               FAKE       FAKE       True      
id0_id6_0009.mp4               FAKE       FAKE       True      
id0_id1_0009.mp4               FAKE       FAKE       True      
id0_id9_0003.mp4               REAL       FAKE       False     
id0_id2_0003.mp4               REAL       FAKE       False     
id0_id2_0008.mp4               FAKE       FAKE       True      
id0_id3_0000.

In [ ]:
# Calcular e exibir acurácia
accuracy = correct / total if total > 0 else 0
print(f"\nAcurácia: {accuracy * 100:.2f}% ({correct}/{total})")


Acurácia: 46.00% (46/100)
